# Convert Osu to Beet Saber

## 1. Convert .osz to .osu

In [2]:
# find install dir
import os
import zipfile

In [3]:
osz_path = '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/'
unzip_path = '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/charts/'

In [4]:
# find all osu zips
oszs = [osz_path+x for x in os.listdir(osz_path) if x.endswith('osz')]
oszs

['/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/killing-my-love.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/ETA.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/stupid-horse.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/duvet.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/running-man.osz']

In [5]:
def retain_file(filepath):
  """Returns if file should be kept
  At time of writing, keeps osu metadata files and mp3 files
  (wav files are used only for hitsounds (irrelevant))"""
  if not (filepath.endswith('osu') or filepath.endswith('mp3')):
    return False
  return True

In [6]:
def keep_file(filepath):
  """
  Tells which files should be kept
  """
  checks = ['easy' in filepath.lower(),
            'normal' in filepath.lower(),
            'hard' in filepath.lower(),
            'insane' in filepath.lower(),
            'extra' in filepath.lower(),
            'expert' in filepath.lower(),
            'audio.mp3' in filepath,
            ]
  # !impt: taiko diffs are like so: kantan, futsuu, muzukashii, oni
  
  return ((True in checks) and ('.wav' not in filepath))

In [7]:
# unzip them. how?
# # make dir per song
# # retain metadata and audio
os.makedirs(unzip_path, exist_ok=True)
for osz in oszs:
  songname = osz.split('/')[-1].split('.')[0]
  save_path = unzip_path+songname+'/'
  with zipfile.ZipFile(osz, 'r') as zip_ref:
    zip_ref.extractall(save_path)

# delete unneeded files:
# # subdirs is minimal directory paths (e.g. killing-my-love, duvet w.o. abspath)
subdirs = [entry for entry in os.listdir(unzip_path) if os.path.isdir(os.path.join(unzip_path, entry))]
for subdir in subdirs:
  dirpath = os.path.join(unzip_path,subdir)
  for file in os.listdir(dirpath):
    if not keep_file(file):
      os.remove(os.path.join(dirpath,file))

## 2. Extract Hit Objects from .osu

In [8]:
# wiki is extremely well documented
# # https://osu.ppy.sh/wiki/en/Client/File_formats/osu_%28file_format%29#type:~:text=Slider%20border%20colour-,Hit%20objects,-Hit%20object%20syntax

In [9]:
example_song = os.path.join(unzip_path,'lovenote')

In [10]:
example_osu_dir = os.listdir(example_song)
example_osu_dir

['audio.mp3']

In [11]:
#example_osu #= example_osu_dir[2]

In [12]:
def classify_object(flag):
  """
  Derive object type from osu's binary flags
  
  Simplied representation:
  - 'circle'
  - 'slider'
  - 'spinner'
  
  """
  # 0 bit indiates slider
  circle = (flag & 1) != 0 # check neq 0 not eq 1 b/c bit shift
  slider = (flag & 1<<1) != 0
  spinner = (flag & 1<<3) != 0
  
  if circle+slider+spinner>1:
    print("Whaaat")
  if circle: 
    return 'circle'
  if slider: 
    return 'slider'
  if spinner: 
    return 'spinner'
  print(f"Something is wrong in object classification. Recieved flag: {flag}")
  return None

A quick note about taiko. Besides naming convention for difficulties, taiko and osu!std objects are stored identically. hype

In [13]:
def extract_hit_objects(osu_path, just_ints = False):
  """
  Find type information and timing data for all hit objects contained in an .osu file
  
  Inputs:
  - osu_path: path to osu metadata file
  
  Returns three zipped arrays
  - note_types: Array of 'slider', 'spinner' and 'circle' strings
  - note_times: Array of integers, representing time in ms when note appears (relative to song start)
  - note_ints: Array of 32 bit integers written in hex containing relevant note info. See game_code/notes.md for more info
  """
  start_str = '[HitObjects]'
  #csv_fields = 'x,y,time,type,hitSound,objectParams,hitSample'.split(',') # from the wiki
  note_types = []
  note_times = []
  note_ints = []
  with open(osu_path, 'r') as map_file:
    content = map_file.read()
    #print(content.find(start_str))
    # see csv_fields comment for how hit objects are parameterized
    hit_objects = content[content.find(start_str)+len(start_str)+1:].split('\n') # extract hit objects
    #print(len(hit_objects))
    # for now, just time and circle/slider class
    for hit_object in hit_objects:
      if len(hit_object)>5: # there will always be at least 6 params
        hit_object_fields = hit_object.split(',')
        #print(hit_object_fields)
        ms, note_type = hit_object_fields[2],hit_object_fields[3]
        note_types.append(classify_object(int(note_type)))
        note_times.append(int(ms))
  # creating note_ints
  for i in range(len(note_types)):
    # !impt
    has_been_hit = 0
    channel = 0 # 0=L, 1=R
    has_repeat_slider = 0
    
    has_slider = note_types[i]=='slider'
    has_spinner = note_types[i]=='spinner'
    has_circle = note_types[i]=='circle'

    combined_int = has_been_hit<<31 | channel<<30| has_spinner<<29 | has_repeat_slider | has_slider<< 27 | has_circle<<26 | note_times[i]

    binary_int = bin(combined_int)[2:].zfill(32)
    #print(binary_int)
    hex_int_blocks = []
    for i in range(8):
      bin4 = binary_int[4*i:4*i+4]
      hex_int_blocks.append(hex(int('0b'+bin4,2))[2:])
    hex_int = '0x'+''.join(hex_int_blocks)
    #print(hex_int)
    note_ints.append(hex_int)
  if just_ints: # programming sin tbh
    return note_ints
  return list(zip(note_types,note_times,note_ints))
  #return pd.read_csv(content)#,usecols=csv_fields)

In [14]:
song = "/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/charts/killing-my-love/LESLIE PARRISH - KILLING MY LOVE (Cut Ver.) (Pincus) [Pepekcz's Hard Ryosuke's RX-7 FC].osu"
hitobjs = extract_hit_objects(song)

In [15]:
# type conversion
print(len(hitobjs))
for i in hitobjs:
  print(i)
  break


328
('slider', 1675, '0x0800068b')


In [16]:
# test deciphering
correct = 0
for a,b,number in hitobjs:
  get_position = int('0b'+'1'*20,2)
  number_bin = int(number,16)
  correct += (get_position&number_bin)==b
correct==len(hitobjs) #if true all good

True

In [17]:
to_c_string = lambda x: '{'+', '.join(x)+'}'

In [18]:
# !impt: where you extract hit objects
duvet_path_easy = '../../oszs/charts/duvet/boa - Duvet (TV Size) (Girls Love) [Easy].osu' 
eta_path_normal = "../../oszs/charts/ETA/NewJeans - ETA (bielsuu) [miss yo's Normal].osu"
killing_my_love_normal = "../../oszs/charts/killing-my-love/LESLIE PARRISH - KILLING MY LOVE (Cut Ver.) (Pincus) [Kirishima-'s Normal Bunta's Impreza].osu"

In [19]:
# add more songs as needed
hitobjs_boa = extract_hit_objects(duvet_path_easy, just_ints=True)
hitobjs_eta = extract_hit_objects(eta_path_normal, just_ints=True)
killing_my_love_normal = extract_hit_objects(killing_my_love_normal, just_ints=True)
# the results
boa_res = to_c_string(hitobjs_boa)
eta_res = to_c_string(hitobjs_eta)
killing_my_love_res = to_c_string(killing_my_love_normal) 

In [20]:
boa_res

'{0x08000776, 0x04000bd1, 0x04000d10, 0x08000eee, 0x080013f0, 0x080017bd, 0x04001e07, 0x04001feb, 0x0400212e, 0x04002312, 0x08002592, 0x04002959, 0x08002a9c, 0x04002f9c, 0x0400317d, 0x0800335f, 0x08003863, 0x04003c26, 0x08003d67, 0x08003fe9, 0x0400464b, 0x0800478d, 0x04004b53, 0x08004dd7, 0x04005235, 0x04005375, 0x08005554, 0x04005a67, 0x08005ba9, 0x080061f3, 0x080065b4, 0x04006a0b, 0x08006bfa, 0x08006fc9, 0x080074de, 0x040078a7, 0x080079e9, 0x0400801d, 0x040082a1, 0x08008525, 0x080088eb, 0x04008f36, 0x08009078, 0x0800943e, 0x08009804, 0x08009a89, 0x0400a088, 0x2000a2f0, 0x0400b734, 0x0800b9b6, 0x0800bd79, 0x0400c279, 0x0400c3ba, 0x0800c63b, 0x0400cb4a, 0x2000cdcc, 0x0800e97b, 0x0400f0f5, 0x0400f38e, 0x0800f4d0, 0x0400f896, 0x0800f9d8, 0x04010022, 0x040102a7, 0x080103e9, 0x080108f1, 0x08010cb7, 0x0801107e, 0x080116c8, 0x04011d13, 0x08011e55, 0x0801221b, 0x04012866, 0x04012aea, 0x08012c26, 0x0801311c, 0x080134e2, 0x200138a8}'

In [25]:
print(len(boa_res.split(',')))

78


In [22]:
eta_res

'{0x08001d05, 0x08001f9f, 0x0800223a, 0x080025b3, 0x0400276f, 0x0400292c, 0x08002ae8, 0x08002d83, 0x0800301d, 0x08003396, 0x04003553, 0x0400370f, 0x080038cc, 0x08003c45, 0x08003fbd, 0x08004336, 0x080046af, 0x0400486c, 0x04004a28, 0x08004b06, 0x08004da1, 0x08005493, 0x0400572d, 0x0400580c, 0x080059c8, 0x04005d41, 0x04005efd, 0x040060ba, 0x08006276, 0x04006511, 0x040065ef, 0x080067ac, 0x04006968, 0x08006b25, 0x08006ce1, 0x0800705a, 0x08007216, 0x080073d3, 0x0800774c, 0x08007908, 0x08007ac5, 0x08007e3d, 0x08007ffa, 0x080081b6, 0x08008373, 0x040088a8, 0x04008986, 0x08008a65, 0x04008c21, 0x04008ddd, 0x08008f9a, 0x04009313, 0x040094cf, 0x0800968c, 0x04009a05, 0x04009bc1, 0x08009d7d, 0x0400a018, 0x0400a0f6, 0x0400a2b3, 0x0800a46f, 0x0400a70a, 0x0400a7e8, 0x0400a9a5, 0x0800ab61, 0x0800aeda, 0x0800b253, 0x0400b5cc, 0x0400b945, 0x0400bcbd, 0x0800c036, 0x0400c1f3, 0x0800c3af, 0x0800c56c, 0x0400c806, 0x0400c8e5, 0x0800caa1, 0x0800ce1a, 0x0800cfd6, 0x0400d193, 0x0800d34f, 0x0800d50c, 0x0400d6c8, 0x

In [24]:
print(len(eta_res.split(',')))

266


In [23]:
killing_my_love_res

'{0x0800068b, 0x0800080a, 0x04000b08, 0x08000c87, 0x08000e06, 0x08001103, 0x04001282, 0x040017be, 0x0800187e, 0x04001cfb, 0x08001e79, 0x04002177, 0x040022f6, 0x08002475, 0x040028f2, 0x08002a71, 0x04002d6e, 0x04002eed, 0x0800306c, 0x040032ab, 0x0800336a, 0x08003668, 0x080038a6, 0x04003ae4, 0x08003c63, 0x04003ea2, 0x08003f61, 0x0800425f, 0x0800449d, 0x040046dc, 0x0800485b, 0x040049dc, 0x04004a9c, 0x08004eb0, 0x08005331, 0x080054b1, 0x04005813, 0x08005933, 0x08005ab3, 0x08005f35, 0x080060b5, 0x04006417, 0x08006537, 0x080066b7, 0x080068f8, 0x08006b39, 0x08006cb9, 0x0400701a, 0x0800713b, 0x080072bb, 0x080074fc, 0x0800773d, 0x080078bd, 0x04007c1e, 0x08007d3f, 0x08007ebf, 0x08008100, 0x08008341, 0x080084c1, 0x080090c5, 0x08009306, 0x08009547, 0x040096c7, 0x04009848, 0x08009908, 0x08009b49, 0x08009cc9, 0x08009f0a, 0x0800a14b, 0x0400a2cb, 0x0400a44c, 0x0800a50c, 0x0800a74d, 0x0800a8cd, 0x0800ab0e, 0x0800ad4f, 0x0400aecf, 0x0400b04f, 0x0800b110, 0x0800b350, 0x0800b4d1, 0x0800b712, 0x0800b952, 0x

In [ ]:
print(len(killing_my_love_res.split(',')))

256
